In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords 
stops = set(stopwords.words("english"))

with open('train.dat','r') as f:
        df1 = pd.DataFrame(l.split("\t") for l in f)
        newcols = {0: 'SentimentClass',1: 'Review',}
        df1.rename(columns=newcols, inplace=True)
        df1["SentimentClass"] = df1["SentimentClass"].apply(lambda x:1 if x=="+1" else -1)

with open('test.dat','r') as f:
        df2 = pd.DataFrame(l for l in f) 
        newcols = {0: 'Review',}
        df2.rename(columns=newcols, inplace=True)
        df2.index = df2.index + 25000

In [ ]:
df3 = pd.DataFrame(df1["Review"])
df4 = pd.DataFrame(df2["Review"])
df5 = df3.append(df4)

In [29]:
#def wordlist(x):
    #x = x.split()
    #stems = []
    #[stems.append(stemmer.stem(word)) for word in x]
    #return stems
vectorizer = TfidfVectorizer(stop_words = 'english',ngram_range=(1, 2), min_df=.0007)
features = vectorizer.fit_transform(df5["Review"])
print features.shape

(50000, 20730)


In [30]:
yu = features[25000:50000].dot(features[0:25000].T)

In [31]:
yu.shape

(25000, 25000)

In [32]:
classification_list=[]
for row in yu:
    Similarity_Scores = {}
    list_of_Scores = []
    inter = row.todense().tolist()
    Similarity_Scores[0] = list(enumerate(inter[0]))
    classification=""
    knnscore=0
    sortedSimilarityScores = sorted(Similarity_Scores[0], key=lambda x: x[1],reverse=True)
    for df_index,cosineSimilarity in sortedSimilarityScores[0:100]:
            if cosineSimilarity==0:
                break
            else:
                weighted_classification = (df1.loc[df_index, 'SentimentClass'])/(cosineSimilarity*cosineSimilarity)    
                knnscore += weighted_classification
    classification_list.append(knnscore)
print ("end of computation")    
with open('format.dat', 'w') as f:
        for item in classification_list:
            if item>0:
                f.write('+1\n')
            elif item ==0:
                roption = ["+1","-1"]
                ranchoice = random.choice(roption)
                f.write(ranchoice+'\n')
            else:
                 f.write('-1\n')
            f.close
print ("end")

end of computation
end
